# Faster R-CNN 구현
### 텐서플로우 기반
### 주석은 한글 위주로 작성

In [1]:
import numpy as np
import os
import glob
import cv2
import xmltodict
import tensorflow as tf
import math
import random
from tqdm import tqdm
from PIL import Image, ImageDraw

## 훈련 이미지 가져오기

In [2]:
train_x_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/train/VOCdevkit/VOC2007/JPEGImages'
train_y_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/train/VOCdevkit/VOC2007/Annotations'

test_x_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/test/VOCdevkit/VOC2007/JPEGImages'
test_y_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/test/VOCdevkit/VOC2007/Annotations'

In [3]:
list_train_x = sorted([x for x in glob.glob(train_x_path + '/**')])    
list_train_y = sorted([x for x in glob.glob(train_y_path + '/**')]) 

list_test_x = sorted([x for x in glob.glob(test_x_path + '/**')])    
list_test_y = sorted([x for x in glob.glob(test_y_path + '/**')]) 

print(len(list_train_x))
print(len(list_test_x))

5011
4952


훈련용 이미지는 5011개, 테스트용 이미지는 4952개가 있음을 알 수 있다. 그리고 둘다 x, y값이 존재한다. 

## RPN - 앵커 준비
#### 입력 이미지 기준으로 앵커를 생성한다.
#### 풀링을 3번 하므로 2^3 = 8이니 (8, 8)부터 (16, 8)...등 8픽셀식 중심 좌표를 옮겨가며 앵커들을 k개씩 생성한다
#### 생성한 앵커들 중 사용할 가치가 있는 앵커를 걸러낸다(이미지 범위를 벗어나지 않는 앵커들만 선정)
#### 선정한 앵커 중 실제 물체의 box와 얼마나 곂치는지(IoU) 계산해본다. 확실히 겹친다 하는 애들을 Positive 앵커로, 거의 안겹친다 하는 애들은 Negataive 앵커로 선정한다. 애매한 애들은 거른다.
#### 이렇게 생성된 앵커들로 미니배치를 생성한다. Positive 128개, Negative 128개로 만드는게 ideal한 구성이긴 한데 Positive한 앵커가 별로 없다. 그래서 Positive를 128개 못채웠으면 Negataive 앵커로 채워준다. 

### 함수로 만드는 이유?
#### 이미지 별로 라벨에서 박스 좌표랑 객체 종류 뽑아내서 loss를 계산해야한다. 그래서 코드의 간결화?를 위해 함수를 만드는 것

In [4]:
# 앵커 생성 함수. 
def make_anchor(anchor_size, anchor_aspect_ratio) :
    # 입력 이미지(그래봤자 224*224긴 하지만)에 맞춰 앵커를 생성해보자 

    anchors = [] # [x,y,w,h]로 이루어진 리스트 
    valid_anchor_bool = [] # 이 앵커를 훈련에 쓸건가? 각 앵커별로 사용 여부를 나타낸다. 

    # 앵커 중심좌표 간격
    interval_x = 16
    interval_y = 16
    Center_max_x = 208 # 224 - 16, 중심좌표가 224가 될 수는 없다.
    Center_max_y = 208 # 224 - 16

    # 2단 while문 생성
    x = 8
    y = 8
    index_count = 0
    while(y <= 224): # 8~208 = 14개 
        while(x <= 224): # 8~208 = 14개 
            # k개의 앵커 생성. 여기서 k = len(anchor_size) * len(anchor_aspect_ratio)다
            for i in range(0, len(anchor_size)) : 
                for j in range(0, len(anchor_aspect_ratio)) :
                    anchor_width = anchor_aspect_ratio[j][0] * anchor_size[i]
                    anchor_height = anchor_aspect_ratio[j][1] * anchor_size[i]

                    anchor = [x, y, anchor_width, anchor_height]
                    anchors.append(anchor)
                    # 앵커가 이미지 경계선을 넘나드나? 필터링
                    if((x - (anchor_width/2) >= 0) and (y - (anchor_height/2) >= 0) and
                    (x + (anchor_width/2) <= 224) and (y + (anchor_height/2) <= 224)):
                        # 경계 안에 있으면 1
                        valid_anchor_bool.append(1)
                    else :
                        valid_anchor_bool.append(0)
            x = x + interval_x 
        y = y + interval_y
        x = 8
    return np.array(anchors), np.array(valid_anchor_bool) # 넘파이로 반환

#### Loss 사용을 위해 Class list도 만들자.
#### 논문의 loss 함수를 보면 pi가 있다. pi는 리스트인데 PASCAL VOC에 존재하는 객체들이 몇 %의 확률로 있느냐 나타내는거다
#### 이를 위해 어떤 객체들이 PASCAL VOC에 존재하는지 알아야한다. 

In [5]:
# 이미지 하나에 대한 라벨값(어떤 클래스가 있는지, 그 클래스는 어떤 박스를 갖고 있는지)
def get_label_fromImage(xml_file_path, Classes_inDataSet): # xml_file_path은 파일 하나의 경로를 나타낸다

    f = open(xml_file_path)
    xml_file = xmltodict.parse(f.read()) 

    # 우선 원래 이미지 크기를 얻는다. 왜냐하면 앵커는 224*224 기준으로 만들었는데 원본 이미지는 224*224가 아니기 때문.
    # 224*224에 맞게 줄일려고 하는거다
    Image_Height = float(xml_file['annotation']['size']['height'])
    Image_Width  = float(xml_file['annotation']['size']['width'])


    Classes_list = [] 
    Ground_Truth_Box_list = [] 
    class_label_list = [] # 원-핫 인코딩으로 만든 y값. 이건 Fast R-CNN 학습에 쓰인다.

    # multi-objects in image
    try:
        for obj in xml_file['annotation']['object']:
            obj_class = obj['name'].lower() 
            # 박스 좌표(왼쪽 위, 오른쪽 아래) 얻기
            x_min = float(obj['bndbox']['xmin']) 
            y_min = float(obj['bndbox']['ymin'])
            x_max = float(obj['bndbox']['xmax']) 
            y_max = float(obj['bndbox']['ymax'])

            # 224*224에 맞게 변형시켜줌
            x_min = float((224/Image_Width)*x_min)
            y_min = float((224/Image_Height)*y_min)
            x_max = float((224/Image_Width)*x_max)
            y_max = float((224/Image_Height)*y_max)

            Ground_Truth_Box = [x_min, y_min, x_max, y_max] 

            index = Classes_inDataSet.index(obj_class) 

            Classes_list.append(index)
            Ground_Truth_Box_list.append(Ground_Truth_Box)

    # single-object in image
    except TypeError as e : 
        
        obj_class = xml_file['annotation']['object']['name'] 
        # 박스 좌표(왼쪽 위, 오른쪽 아래) 얻기
        x_min = float(xml_file['annotation']['object']['bndbox']['xmin']) 
        y_min = float(xml_file['annotation']['object']['bndbox']['ymin']) 
        x_max = float(xml_file['annotation']['object']['bndbox']['xmax']) 
        y_max = float(xml_file['annotation']['object']['bndbox']['ymax']) 

        # 224*224에 맞게 변형시켜줌
        x_min = float((224/Image_Width)*x_min)
        y_min = float((224/Image_Height)*y_min)
        x_max = float((224/Image_Width)*x_max)
        y_max = float((224/Image_Height)*y_max)

        Ground_Truth_Box = [x_min, y_min, x_max, y_max]  

        index = Classes_inDataSet.index(obj_class) 

        Classes_list.append(index)
        Ground_Truth_Box_list.append(Ground_Truth_Box)


    for i in range(0, len(Classes_list)):
        one_hot = []
        for j in range(0, len(Classes_inDataSet) + 1): # k + 1 클래스(사물 + 배경)를 원-핫 인코딩
            if Classes_list[i] == j :
                one_hot.append(1)
            
            else :
                 one_hot.append(0)
        class_label_list.append(one_hot)


    return np.array(class_label_list), np.array(Ground_Truth_Box_list)


In [6]:
def get_iou(anchor, Ground_Truth_Box_List) : # anchor, ground truth box list를 받아 각 IOU을 계산하고 제일 큰걸 반환

    IoU_max = float(0.0)
    index_ground_truth_box = -1
    for i in range(0, len(Ground_Truth_Box_List)):

        ground_truth_box = Ground_Truth_Box_List[i]

        InterSection_min_x = max(anchor[0], ground_truth_box[0])
        InterSection_min_y = max(anchor[1], ground_truth_box[1])

        InterSection_max_x = min(anchor[2], ground_truth_box[2])
        InterSection_max_y = min(anchor[3], ground_truth_box[3])

        InterSection_Area = 0

        if (InterSection_max_x - InterSection_min_x + 1) >= 0 and (InterSection_max_y - InterSection_min_y + 1) >= 0 :
            InterSection_Area = (InterSection_max_x - InterSection_min_x + 1) * (InterSection_max_y - InterSection_min_y + 1)

        box1_area = (anchor[2] - anchor[0]) * (anchor[3] - anchor[1])
        box2_area = (ground_truth_box[2] - ground_truth_box[0]) * (ground_truth_box[3] - ground_truth_box[1])
        Union_Area = box1_area + box2_area - InterSection_Area

        IoU = (InterSection_Area/Union_Area)
        if IoU > IoU_max :
            IoU_max = IoU
            index_ground_truth_box = i

    return IoU_max, index_ground_truth_box # 어떤 박스와 IoU가 제일 높았는지
    

In [106]:
# 앵커들을 Positive, Negative 앵커로 나누자
def align_anchor(anchors, valid_anchor_bool, Ground_Truth_Box_list):

    # 각 앵커는 해당 위치에서 구한 여러가지 Ground truth Box와의 ioU 중 제일 높은거만 가져온다. 
    IoU_List = np.array([])
    index_ground_truth_box_list = []
    for i in range(0, len(anchors)):
        anchor_minX = anchors[i][0] - (anchors[i][2]/2)
        anchor_minY = anchors[i][1] - (anchors[i][3]/2)
        anchor_maxX = anchors[i][0] + (anchors[i][2]/2)
        anchor_maxY = anchors[i][1] + (anchors[i][3]/2)

        anchor = [anchor_minX, anchor_minY, anchor_maxX, anchor_maxY]
        IoU, index_ground_truth_box = get_iou(anchor, Ground_Truth_Box_list)
        IoU_List = np.append(IoU_List, IoU)
        index_ground_truth_box_list.append(index_ground_truth_box)

    # positive, negative 앵커 분류
    for i in range(0, 14*14):
        # 각 위치에서 IoU가 가장 큰 앵커 or IoU가 0.7 넘는 앵커를 positive앵커로 한다. 
        for num in range(0, 9):
            index = 9 * i + num
            IoU_inOneSpot = IoU_List[9 * i : 9 * i + 9]

            maxIoU_inOneSpot = max(IoU_inOneSpot)
            if IoU_List[index] < 0.3 : # negative anchor
                valid_anchor_bool[index] = 1
            elif maxIoU_inOneSpot == IoU_List[index] or IoU_List[index] > 0.7 : # positive anchor
                valid_anchor_bool[index] = 2
            else: # 애매한 앵커들
                valid_anchor_bool[index] = 0


    return valid_anchor_bool, np.array(index_ground_truth_box_list) # 모든 앵커에 대한 ground truth box list. 앖으면 -1

### 훈련을 위한 미니배치 256(positive 128, negative 128개) 선발

In [116]:
def Create_Minibatch(anchors, index_ground_truth_box_list, valid_anchor_bool) : 

    anchors_forMiniBatch = np.array([])
    index_ground_truth_box_list_forMiniBatch = np.array([])
    index_pos = np.array([]) # 긍정 앵커의 인덱스
    index_neg = np.array([]) # 부정 앵커의 인덱스
    output_index_forMiniBatch = np.array([])
    for i in range(0, len(valid_anchor_bool)):
        if valid_anchor_bool[i] == 2:
            index_pos = np.append(index_pos, i)
        elif valid_anchor_bool[i] == 1:
            index_neg = np.append(index_neg, i)

    max_for = min([128, len(index_pos)])

    ran_list = random.sample(range(0, len(index_pos)), max_for)
    for i in range(0, len(ran_list)) :
        pos_index = int(index_pos[ran_list[i]])
        anchors_forMiniBatch = np.append(anchors_forMiniBatch, anchors[pos_index])
        output_index_forMiniBatch = np.append(output_index_forMiniBatch, pos_index)
        index_ground_truth_box_list_forMiniBatch = np.append(index_ground_truth_box_list_forMiniBatch, index_ground_truth_box_list[pos_index])

    # positive Anchor가 128개 미만일 경우 negative anchor에서 빈걸 채워줌
    ran_list = random.sample(range(0, len(index_neg)), 256 - max_for) # 랜덤성 증가?를 위해 또다시 난수 생성
    for i in range(0, len(ran_list)) :
        neg_index = int(index_neg[[ran_list[i]]])
        anchors_forMiniBatch = np.append(anchors_forMiniBatch, anchors[neg_index])
        output_index_forMiniBatch = np.append(output_index_forMiniBatch, neg_index)

    anchors_forMiniBatch = np.reshape(anchors_forMiniBatch, (-1,4))

    return anchors_forMiniBatch, index_ground_truth_box_list_forMiniBatch, output_index_forMiniBatch, max_for # 미니배치와 어디서부터 negative 앵커가 시작되는지

### Loss 함수 생성

### RPN의 Output을 Loss에 쓰기 위해 필터링

In [111]:
def get_output_anchor_forLoss(cls_layer_output, reg_layer_output, output_index_forMiniBatch): # 가공한 출력값 중 미니배치에 맞춰 256개 선발

    cls_layer_output_forMiniBatch = np.array([])
    reg_layer_output_forMiniBatch = np.array([])

    cls_layer_output = np.reshape(cls_layer_output, (-1, 2))
    reg_layer_output = np.reshape(reg_layer_output, (-1, 4))

    for i in range(0, len(output_index_forMiniBatch)):
        index = int(output_index_forMiniBatch[i])
        cls_layer_output_forMiniBatch = np.append(cls_layer_output_forMiniBatch, cls_layer_output[index])
        reg_layer_output_forMiniBatch = np.append(reg_layer_output_forMiniBatch, reg_layer_output[index])
        
    cls_layer_output_forMiniBatch = np.reshape(cls_layer_output_forMiniBatch, (-1, 2))
    reg_layer_output_forMiniBatch = np.reshape(reg_layer_output_forMiniBatch, (-1, 4))


    return cls_layer_output_forMiniBatch, reg_layer_output_forMiniBatch

### Loss 계산을 위한 함수

In [10]:
def Smooth_L1(ti, ti_star) :
    difference_ti = ti - ti_star
    smooth_L1 = 0
    if abs(difference_ti) < 1: smooth_L1 = 0.5 * (difference_ti * difference_ti)
    else : smooth_L1 = abs(difference_ti) - 0.5

    return smooth_L1

In [118]:
# Loss_Regression. Lreg에 해당
def Loss_Regression(predict_box, anchor_box, groundTruth_box) : 
    groundTruth_box = np.array([(groundTruth_box[2] + groundTruth_box[0])/2, (groundTruth_box[1] + groundTruth_box[3])/2, groundTruth_box[2] - groundTruth_box[0], groundTruth_box[3] - groundTruth_box[1]])

    t_x = (predict_box[0] - anchor_box[0])/anchor_box[2]
    t_y = (predict_box[1] - anchor_box[1])/anchor_box[3]
    t_w = math.log10(predict_box[2]/anchor_box[2])
    t_h = math.log10(predict_box[3]/anchor_box[3])


    t_x_star = (groundTruth_box[0] - anchor_box[0])/anchor_box[2]
    t_y_star = (groundTruth_box[1] - anchor_box[1])/anchor_box[3]
    t_w_star = math.log10(groundTruth_box[2]/anchor_box[2])
    t_h_star = math.log10(groundTruth_box[3]/anchor_box[3])

    # Smooth L1 구하기
    # 구성요소가 4개니까 4번 구해야겠지?
    smooth_L1_x = Smooth_L1(t_x, t_x_star)
    smooth_L1_y = Smooth_L1(t_y, t_y_star)
    smooth_L1_w = Smooth_L1(t_w, t_w_star)
    smooth_L1_h = Smooth_L1(t_h, t_h_star)

    smooth_L1_list = smooth_L1_x + smooth_L1_y + smooth_L1_w + smooth_L1_h

    return smooth_L1_list # 모아서 반환


In [12]:
# L_cls
def Loss_Classes(cls_layer_output, pi_ground_truth) :
    log_loss = -pi_ground_truth * math.log10(cls_layer_output[0]) - (1-pi_ground_truth)*math.log10(1-cls_layer_output[0])
    return log_loss

### 생각나는거
### 테스트할 때 RPN에서 나온 RoI들을 Object score 기준으로 나열한 다음 상위 몇개만 걸러낸다(NMS)
### 즉, 스코어를 뽑아낼 수 있는 방향으로 RPN의 Loss를 만들어야한다.

In [113]:
# Loss함수 수행. RPN의 출력값, 입력 이미지의 라벨값, 앵커를 받는다.
def Loss_RPN(cls_layer_output, reg_layer_output, anchors, valid_anchor_bool, Ground_Truth_Box_list):
    
    # 앵커 선별
    valid_anchor_bool, index_ground_truth_box_list = align_anchor(anchors, valid_anchor_bool, Ground_Truth_Box_list) # pos, neg 앵커 표시 + 모든 앵커에 대한 ground_truth_box_list
    anchors_forMiniBatch, index_ground_truth_box_list_forMiniBatch, output_index_forMiniBatch, max_for = Create_Minibatch(anchors, index_ground_truth_box_list, valid_anchor_bool)
    # output 선별 
    cls_layer_output_forMiniBatch, reg_layer_output_forMiniBatch = get_output_anchor_forLoss(cls_layer_output, reg_layer_output, output_index_forMiniBatch)

    # 자, 여기서 잠깐 생각을 해보자
    # k = 9로 했으니 cls_layer의 결과값은 18개, reg_layer의 결과값은 36개가 있다. 
    # 앵커를 256개 랜덤해서 뽑겠다는 말이니까 결과값도 256쌍을 뽑아야한다. 그렇지? 이 256쌍은 뽑힌 앵커에 해당하는 출력값을 뽑아야한다. 

    N_cls = len(anchors_forMiniBatch)
    N_reg = 14*14 # VGG 특성맵 최종 output 넓이. 
    lambda_forLoss = 10

    Lcls_sum = 0
    Lreg_sum = 0

    # RPN의 cls output이 pi, reg  output이 ti이다.
    # k = 9일 때 pi는 18개, ti는 36개라는 말.
    for i in range(0, len(anchors_forMiniBatch)) :
        pi_ground_truth = 0
        if i < max_for : # Positive Anchor
            pi_ground_truth = 1
        #Lcls(classes loss)
        Lcls_sum = Lcls_sum + Loss_Classes(cls_layer_output_forMiniBatch[i], pi_ground_truth)

        #Lreg(regression loss)
        if i < max_for : 
            index = int(index_ground_truth_box_list_forMiniBatch[i])
            Ground_Truth_Box = Ground_Truth_Box_list[index]
            Lreg_sum = Lreg_sum + Loss_Regression(reg_layer_output_forMiniBatch[i], anchors_forMiniBatch[i], Ground_Truth_Box)

    loss = (1/N_cls) * Lcls_sum + lambda_forLoss*(1/N_reg)*Lreg_sum

    return loss

## Fast-RCNN 부분(RoIPooling 등)

## 훈련을 위한 함수

In [14]:
# 입력용 이미지 생성. 224, 224로 변환시키고 채널 값(0~255)를 0~1 사이의 값으로 정규화 시켜줌
def make_input(image_file_list): 
    images_list = []
    
    for i in tqdm(range(0, len(image_file_list))) :
        
        image = cv2.imread(image_file_list[i])
        image = cv2.resize(image, (224, 224))/255
        
        images_list.append(image)
    
    return np.asarray(images_list)

In [119]:
def get_output_RPN(image, SharedConvNet, RPN_intermediate_layer, RPN_cls_Layer, RPN_reg_Layer):
    Conv_FeatureMap = SharedConvNet(np.expand_dims(image, axis=0))
    RPN_intermedi = RPN_intermediate_layer(Conv_FeatureMap)

    cls_layer_output = RPN_cls_Layer(RPN_intermedi)[0] # 14*14*18
    reg_layer_output = RPN_reg_Layer(RPN_intermedi)[0] # 14*14*36

    cls_layer_output_forUse = []
    reg_layer_output_forUse = []

    anchor_size = [32, 64, 128] # 이미지 크기가 224*224라 32, 64, 128로 지정
    anchor_aspect_ratio = [[1,1],[1,0.5], [0.5,1]] # W*L기준 

    for y in range(0, 14):
        for x in range(0, 14):
                for i in range(0, len(anchor_size)):
                    for j in range(0, len(anchor_aspect_ratio)):
                        score = [cls_layer_output[y][x][2*(3*i+j)], cls_layer_output[y][x][2*(3*i+j) + 1]] # 오브젝트다 vs 아니다 2개 항목에 대한 스코어 저장(각 앵커당)
                        cls_layer_output_forUse.append(tf.nn.softmax(score))
                        
                        center_x = 8 + 16 * x 
                        center_y = 8 + 16 * y
                        w = anchor_size[i] * anchor_aspect_ratio[j][0]
                        h = anchor_size[i] * anchor_aspect_ratio[j][1]

                        score_x = center_x + reg_layer_output[y][x][4*(3*i+j)]
                        score_y = center_y + reg_layer_output[y][x][4*(3*i+j) + 1]
                        score_w = w + reg_layer_output[y][x][4*(3*i+j) + 2]
                        score_h = h + reg_layer_output[y][x][4*(3*i+j) + 3]

                        box_info = [score_x, score_y, score_w, score_h]
                        reg_layer_output_forUse.append(box_info)

    return np.array(cls_layer_output_forUse), np.array(reg_layer_output_forUse)

In [122]:
def training_RPN(input_image, xml_file, Classes_inDataSet, SharedConvNet, RPN_intermediate_layer, RPN_cls_Layer, RPN_reg_Layer, anchors, valid_anchor_bool, lr):
    Classes_inImage, Ground_Truth_Box_list = get_label_fromImage(xml_file, Classes_inDataSet) # label 휙득
    cls_layer_output_forUse, reg_layer_output_forUse = get_output_RPN(input_image, SharedConvNet, RPN_intermediate_layer, RPN_cls_Layer, RPN_reg_Layer) # output 구하기
    loss = Loss_RPN(cls_layer_output_forUse, reg_layer_output_forUse, anchors, valid_anchor_bool, Ground_Truth_Box_list)

    # loss를 구했는데 어떻게 훈련시키냐 아 ㅋㅋ
    momentum = 0.9
    weight_decay = 0.0005


### RPN 흐름 정리(훈련)

In [16]:
# 파일 리스트
image_file_list = sorted([x for x in glob.glob(train_x_path + '/**')])
xml_file_list = sorted([x for x in glob.glob(train_y_path + '/**')])

In [17]:
# 입력 이미지 생성
input_image_list = make_input(image_file_list)

100%|██████████| 5011/5011 [00:17<00:00, 286.51it/s]


In [18]:
# 존재하는 객체 종류를 알아내자
def get_Classes_inImage(xml_file_list):
    classes = []

    for xml_file_path in xml_file_list: 

        f = open(xml_file_path)
        xml_file = xmltodict.parse(f.read())
        # 사진에 객체가 여러개 있을 경우
        try: 
            for obj in xml_file['annotation']['object']:
                classes.append(obj['name'].lower()) # 들어있는 객체 종류를 알아낸다
        # 사진에 객체가 하나만 있을 경우
        except TypeError as e: 
            classes.append(xml_file['annotation']['object']['name'].lower()) 
        f.close()

    classes = list(set(classes)) # set은 중복된걸 다 제거하고 유니크한? 아무튼 하나만 가져온다. 그걸 리스트로 만든다
    classes.sort() # 정렬

    return classes

In [19]:
Classes_inDataSet = get_Classes_inImage(xml_file_list)

In [20]:
max_num = len(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers) # 레이어 최대 개수

SharedConvNet = tf.keras.models.Sequential()
for i in range(0, max_num-1):
    SharedConvNet.add(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers[i])

initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)

RPN_intermediate_layer = tf.keras.layers.Conv2D(512, (3,3), kernel_initializer = initializer, activation='relu', padding='SAME', input_shape=(14, 14, 512))
RPN_cls_Layer = tf.keras.layers.Conv2D(18, (1,1), kernel_initializer = initializer, input_shape=(14, 14, 512)) # 클래스인가? 아닌가? 
RPN_reg_Layer = tf.keras.layers.Conv2D(36, (1,1), kernel_initializer = initializer, input_shape=(14, 14, 512))

In [149]:
test = (RPN_cls_Layer.weights[0][0][0][511][17])

In [157]:
test + 2 # 가중치를 일일이 다 불러와서 경사하강법 적용하면 되겠다 ^오^
# 가중치 불러오기 -> 업데이트 -> 저장

<tf.Tensor: shape=(), dtype=float32, numpy=1.9944729>

In [143]:
print(test)

[<tf.Variable 'conv2d_1/kernel:0' shape=(1, 1, 512, 18) dtype=float32, numpy=
array([[[[ 0.00153891, -0.00039054, -0.00392993, ..., -0.00724896,
           0.00945011,  0.00443572],
         [-0.00159282,  0.00469282, -0.01365773, ..., -0.00591567,
           0.00327307, -0.00834619],
         [ 0.00762109,  0.017832  , -0.00866503, ..., -0.01427393,
          -0.01806705,  0.00723966],
         ...,
         [-0.00191052,  0.01792054, -0.00434975, ...,  0.00578483,
          -0.01136053,  0.0007477 ],
         [ 0.01468072, -0.00609638,  0.00692405, ..., -0.01397917,
           0.01846595,  0.01325233],
         [ 0.00669655,  0.00203889,  0.0202539 , ...,  0.01167233,
          -0.0087166 , -0.00552713]]]], dtype=float32)>, <tf.Variable 'conv2d_1/bias:0' shape=(18,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.], dtype=float32)>, <tf.Variable 'conv2d_1/kernel:0' shape=(1, 1, 512, 18) dtype=float32, numpy=
array([[[[ 0.00153

In [26]:
n = 3 # 논문에서 n은 VGG16을 거치고 나온 특성맵 위를 슬라이딩 할 커널의 크기

# 생성할 앵커 크기는? 
anchor_size = [32, 64, 128] # 이미지 크기가 224*224라 32, 64, 128로 지정
anchor_aspect_ratio = [[1,1],[1,0.5], [0.5,1]] # W*L기준 
anchors, valid_anchor_bool = make_anchor(anchor_size, anchor_aspect_ratio) # 앵커 생성 + 유효한 앵커 인덱스 휙득

